In [1]:
import re
import pandas as pd
import numpy as np
import sys

#sys.

In [34]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = "eLTHNVGuK3e6ZDDENKJD9k6w8"
consumer_secret = "UbJDVAj75efcWQ1mbVVrbFH10DbdewJhV74HPu9BfMm7Uafe6T"
access_key = "118637669-Xce1sDf9KRVlBlarMCZJ9PX51mJJ2OImdVtVwtyl"
access_secret = "r9D9yhxoJCai6clYfOcVEp1HQVdhxZD6wDQ9ZXeMwKcWd"

NoOfTweets=900 # May be assuming 5 per day; would get 6 months data


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=NoOfTweets)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print ("getting tweets before %s" % (oldest))
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=NoOfTweets,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print ("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    
    '''
    #write the csv	
    with open('%s_tweets.csv' % screen_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    
    pass
    '''
    return outtweets

if __name__ == '__main__':
    #pass in the username of the account you want to download
    outtweets = get_all_tweets("wipro")

getting tweets before 846013926886309887
...400 tweets downloaded so far
getting tweets before 841046164925501439
...600 tweets downloaded so far
getting tweets before 837350299513782272
...800 tweets downloaded so far
getting tweets before 831231734930890756
...1000 tweets downloaded so far
getting tweets before 825274753673998335
...1200 tweets downloaded so far
getting tweets before 821159870938411008
...1400 tweets downloaded so far
getting tweets before 815067469689786367
...1600 tweets downloaded so far
getting tweets before 808017015491465219
...1800 tweets downloaded so far
getting tweets before 801198600478978047
...2000 tweets downloaded so far
getting tweets before 794542503911272447
...2200 tweets downloaded so far
getting tweets before 789126577116237827
...2400 tweets downloaded so far
getting tweets before 783200081319112704
...2600 tweets downloaded so far
getting tweets before 776926188325380096
...2800 tweets downloaded so far
getting tweets before 770984564617465855


In [2]:
import csv, codecs, cStringIO

class UTF8Recoder:
    """
    Iterator that reads an encoded stream and reencodes the input to UTF-8
    """
    def __init__(self, f, encoding):
        self.reader = codecs.getreader(encoding)(f)

    def __iter__(self):
        return self

    def next(self):
        return self.reader.next().encode("utf-8")

class UnicodeReader:
    """
    A CSV reader which will iterate over lines in the CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        f = UTF8Recoder(f, encoding)
        self.reader = csv.reader(f, dialect=dialect, **kwds)

    def next(self):
        row = self.reader.next()
        return [unicode(s, "utf-8") for s in row]

    def __iter__(self):
        return self

class UnicodeWriter:
    """
    A CSV writer which will write rows to CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        # Redirect output to a queue
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode("utf-8") for s in row])
        # Fetch UTF-8 output from the queue ...
        data = self.queue.getvalue()
        data = data.decode("utf-8")
        # ... and reencode it into the target encoding
        data = self.encoder.encode(data)
        # write to the target stream
        self.stream.write(data)
        # empty queue
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


with open('wipro_tweets.csv', 'wb') as f:
    writer = csv.writer(f)
    #writer = UnicodeWriter(f)
    writer.writerow(["id","created_at","text"])
    writer.writerows(outtweets)


NameError: name 'outtweets' is not defined

In [112]:
import pandas as pd

wiproTweets = pd.read_csv('wipro_tweets_hash.csv')


import re

'''
myre = re.compile(u'['
                  u'\U0001F300-\U0001F64F'
                  u'\U0001F680-\U0001F6FF'
                  u'\u2600-\u26FF\u2700-\u27BF]+', 
                  re.UNICODE)

'''
#i = myre.sub('', u'Some example text with a sleepy face: \U0001f62a')
 
def isNaN(num):
    return num != num
#text.replaceAll("[^\\u009d]", "\uFFFD");
#for index,i in enumerate(wiproTweets['hashtags']):
    #i = myre.replaceAll('', i)
#    if (isNaN(i)):
#        i = " "
#    print (index,i)
#    print (i.decode("utf-8").strip())

def handleNan(num):
    if (isNaN(num)):
        return ""
    else:
        return num
wiproTweets['hashtags'] = wiproTweets['hashtags'].apply(lambda x: handleNan(x))

In [82]:
#wiproTweets
def getwordStartingWithpatternInSentence(sentence, pattern):
    return " ".join([word for word in sentence.split() if word.startswith(pattern)])


#print (getwordStartingWithpatternInSentence("Hey Sahil #Wipro is great #company"
#      ,'#'))

In [86]:
#wiproTweets
#wiproTweets['hashtags'] = wiproTweets['text'].apply(lambda x: getwordStartingWithpatternInSentence(x, '#'))

#wiproTweets.to_csv("wipro_tweets_hash.csv")

In [113]:

wiproTweets['text'] = wiproTweets['text'].map(lambda words: re.sub(r'\w*\d\w*', '', str(words)).strip())
wiproTweets['text'] = wiproTweets['text'].map(lambda words: re.sub(r'http://.', '', str(words)).strip())
wiproTweets['text'] = wiproTweets['text'].map(lambda words: re.sub(r'https://.', '', str(words)).strip())
wiproTweets['text'] = wiproTweets['text'].map(lambda words: re.sub(r'â€¦', '', str(words)).strip())
wiproTweets['text'] = wiproTweets['text'].map(lambda words: re.sub(r'co/', '', str(words)).strip())

In [114]:
#wiproTweets['text'] = wiproTweets['text'].apply(lambda x: x.decode('utf-8').strip())

#for i in wiproTweets['text']:
#    print (i)
#    print (i.decode("utf-8").strip())

In [122]:
# http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-topics-extraction-with-nmf-lda-py

wiproTweets = wiproTweets.iloc[0:2630]
wiproTweetsFiltered = wiproTweets[wiproTweets['hashtags'] != ""]

In [123]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
cntvectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                stop_words='english')

Extracting tf features for LDA...


In [124]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(wiproTweetsFiltered['hashtags'])
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(wiproTweetsFiltered['hashtags'])
print("done in %0.3fs." % (time() - t0))

# Fit the NMF model
print("Fitting the NMF model with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

done in 0.077s.
Extracting tf features for LDA...
done in 0.044s.
Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.446s.


In [125]:
tf.getmaxprint()

50

In [129]:
for t in tf[1]:
    print (t.data)

[1]


In [127]:
from operator import itemgetter
import sys
 
# maps words to their counts
word2count = {}
 
# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    # convert count (currently a string) to int
    try:
        count = int(count)
        word2count[word] = word2count.get(word, 0) + count
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        pass
 
# sort the words lexigraphically;
# this step is NOT required, we just do it so that our
# final output will look more like the official Hadoop
# word count examples
sorted_word2count = sorted(word2count.items(), key=itemgetter(0))
 
# write the results to STDOUT (standard output)
for word, count in sorted_word2count:
    print ('%s\t%s'% (word, count))


In [130]:
#print("\nTopics in NMF model:")
#tfidf_feature_names = cntvectorizer.get_feature_names()
#print_top_words(nmf, tfidf_feature_names, n_top_words)
import numpy as np

print (tf.shape)
print (tf.A[0,:].shape,tf.A[1,:].shape)
print()

print(tf.indices.shape)
print (len(set(tf.indices)))

word_count = {}
tfidf_count = {}
for index,str in enumerate(tf_vectorizer.get_feature_names()):
    #word_count+={str:np.sum(tf.A[:,index])}
    word_count[str] = np.sum(tf.A[:,index])
    
for index,str in enumerate(tfidf_vectorizer.get_feature_names()):
    #word_count+={str:np.sum(tf.A[:,index])}
    tfidf_count[str] = np.sum(tfidf.A[:,index])

(2385, 587)
(587L,) (587L,)

(4159L,)
587


In [131]:
import operator
#x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(word_count.items(), key=operator.itemgetter(1))
sorted_hash = sorted(tfidf_count.items(), key=operator.itemgetter(1))


In [132]:
sorted_hash[-200:]

[(u'promax', 3.3879331711916825),
 (u'wiproeot', 3.4268163893709307),
 (u'tco', 3.4357972302303792),
 (u'automated', 3.4402889826657232),
 (u'mobilefirst', 3.4477857390886841),
 (u'rpa', 3.4641174786324811),
 (u'managedservices', 3.475402238106271),
 (u'artificialintelligence', 3.5006171788458471),
 (u'businesstransformation', 3.5349316602645997),
 (u'connecteddevices', 3.5357340867354523),
 (u'dataanalytics', 3.5446520727374637),
 (u'gartnersec', 3.5567148516808),
 (u'turnkey', 3.5761027950919444),
 (u'mobileindustry', 3.5886970407244316),
 (u'cognitivecomputing', 3.5958793863702914),
 (u'cloudworld', 3.6084749067919852),
 (u'student', 3.6221628394433845),
 (u'retailers', 3.625054209568459),
 (u'ittransformation', 3.646207447805395),
 (u'datamigration', 3.6716488036805783),
 (u'digitalera', 3.67730159201595),
 (u'customertestimonial', 3.6877884633117683),
 (u'customersatisfaction', 3.7010002942774918),
 (u'engineering', 3.701831306476385),
 (u'demonetization', 3.7275119610034526),
 (u

In [133]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Convert all the required text into a single string here 
#and store them in word_string

#you can specify fonts, stopwords, background color and other options

wordcloud = WordCloud(font_path='C:/Users/Dikshit/Desktop/last/VLC/skins/fonts/FreeSans.ttf',
                          stopwords=STOPWORDS,
                          background_color='white',
                          width=1200,
                          height=1000
                         ).generate(wiproTweetsFiltered['hashtags'])


plt.imshow(wordcloud)
plt.axis('off')
plt.show()

ImportError: No module named wordcloud

In [ ]:
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
#print_top_words(nmf, tfidf_feature_names, n_top_words)


In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
wiproTweets